In [1]:
import pandas as pd

In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install pyarrow==11.0.0

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.4.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Adding LoRA adapters  in order to update 1 to 10% of all the parameters

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1XamvWYinY6FOSX9GLvnqSjjsNflxdhNc?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

# Using the Alpaca dataset

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")


Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [6]:
df = pd.read_json("hf://datasets/asbjoernrubek/clinical_nutritional_coach_formatted/transcripts_formatted.jsonl", lines=True)

In [ ]:
df.head()

In [7]:
# Reading and loading dataset
df = pd.read_json("hf://datasets/asbjoernrubek/clinical_nutritional_coach_formatted/transcripts_formatted.jsonl", lines=True)
df = df.iloc[:1001,:]

# Sorting out patients' words (labels) and dietitians' responses (outputs)
def processed_data(df):
  inputs = []
  labels = []

  for index, row in df.iterrows():
    conversation = row['conversation']

    conversation_processed = conversation.replace('<s>[INST]', 'patient:').replace('[/INST]','Nutritionist:')

    input_text = conversation_processed.split('Nutritionist:')[0].strip()
    label_text = 'Nutritionist:' + conversation_processed.split('Nutritionist:')[1].strip() if 'Nutritionist:' in conversation_processed else ''

    inputs.append(input_text)
    labels.append(label_text)
  return inputs, labels

inputs, labels = processed_data(df)
print(inputs[:5])
print(labels[:5])

f_labels = []
for label in labels:
  label = label.split('<')[0].split('Nutritionist:')[1]
  f_labels.append(label)

data = pd.DataFrame({'output': f_labels, 'input': inputs, 'instruction': '["Act like a friendly dietician", "A very intelligent patient dietician","Never give dangerous advice"]'})

["patient: I'm facing a lot of issues with managing my daily calories. I’m not sure what my goals should be.", "patient: I've been trying to keep a steady workout routine for a while now, but it feels like I'm just resisting change instead of embracing it. Every time I make a little progress, something comes up, and I fall right back into old habits. It's really frustrating because it's starting to feel like a cycle of failure that I can't seem to break out of. I could use some help to figure out how to make exercise a more consistent part of my life.", "patient: I'm struggling with overcoming my habits of large portions at meals. Can you give me some suggestions?", "patient: I'm really focused on maintaining my current weight, but I find sticking to my goals is tough when it comes to portion sizes. I need some help with that.", "patient: Hi, I'm really focusing on managing my weight and I'm hoping for success, but it's been overwhelming. Any guidance?"]
["Nutritionist:It's really grea

In [ ]:
data.head()

In [8]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)

In [9]:
dataset

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 1001
})

In [ ]:
dataset[:3]

In [10]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/1001 [00:00<?, ? examples/s]

In [ ]:
dataset[:3]

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 12,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1001 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [12]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,001 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 12
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.663600
2,2.632400
3,2.554600
4,2.349300
5,2.217500
6,2.022000
7,1.868500
8,1.598900
9,1.339200
10,1.266200


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

In [ ]:
# get from the user

In [14]:
%%capture
!pip install streamlit

In [15]:
import streamlit as st

In [ ]:
user_input = input("Enter your prompt")

In [16]:
# def get_user_input():
#     # Simulate getting user input in a chatbot environment
#     user_input = "I am having challenges controlling my weight"  # Replace this with actual user input retrieval
#     return user_input

def chatbot_response(user_input):
    # Process the user input and generate a response
    response = f"Thank you for sharing. You mentioned: {user_input}"
    return response


In [18]:
def generate_response(user_input):
  # alpaca_prompt = Copied from above
  FastLanguageModel.for_inference(model) # Enable native 2x faster inference
  inputs = tokenizer(
  [
      alpaca_prompt.format(
          "", # instruction
          user_input, # input
          "", # output - leave this blank for generation!
      )
  ], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
  response = tokenizer.batch_decode(outputs)
  response = response[0].split("Response:")[-1][1:-15]
  return response

In [19]:
generate_response("I am having challenges controlling my weight")

"I understand that you're having difficulty controlling your weight. It's important to recognize that this is a common struggle, and you're not alone. Let's take a moment to reflect on what's working well in your efforts to manage your weight. What strategies or habits have you found helpful so far?"

# Saving, loading finetuned models


In [20]:
saved_model = model.save_pretrained("nutritionist_model") # Local saving
saved_tonizer = tokenizer.save_pretrained("nutritionist_model")

In [ ]:
# !pip install unsloth

In [ ]:
# from joblib import dump, load
# dump(model, 'model_unsloth.joblib')

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# # model_name = model
# # tokenizer_name = tokenizer
# # model = AutoModelForCausalLM.from_pretrained(model_name)
# # tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# def save_model_and_tokenizer(model, tokenizer, model_filename, tokenizer_filename):
#     with open(model_filename, 'wb') as model_file:
#         pickle.dump(model, model_file)

#     with open(tokenizer_filename, 'wb') as tokenizer_file:
#         pickle.dump(tokenizer,tokenizer_file)




In [ ]:
# save_model_and_tokenizer(model,tokenizer,"model.pkl","tokenizer.pkl")

In [ ]:
# import torch

# saving_model(saved_model,"pretrained_model.pkl")

In [ ]:
# def load_model(filename):
#   with open(filename, 'rb') as file:
#     model = pickle.load(file)
#   return model

In [ ]:
# model = load_model("pretrained_model.pkl")


Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
# import pickle
# from google.colab import files
# filename = 'nutritionist_model.pkl'
# with open(filename, 'wb') as file:
#     pickle.dump(model, file)

# # Download the file
# files.download('nutritionist_model.pkl')
#Assuming you have a trained model and tokenizer

# Pickle saving the model

In [21]:
# import pickle
# from google.colab import files
# # # filename = 'nutritionist_model.pkl'
# # # pickle.dump(rf_model,open(filename,'wb'))
# # with open("nutritionist_model.pkl","wb") as f:
# #   pickle.dump("nutritionist_model",f)

# filename = 'nutritionist_model.pkl'
# with open(filename, 'wb') as file:
#     pickle.dump(model, file)

# # Download the file
# files.download('nutritionist_model.pkl')
# Assuming you have a trained model and tokenizer

import shutil
from google.colab import files

# Compress the directory into a zip file
shutil.make_archive("nutritionist_model", 'zip', "nutritionist_model")

# Download the zip file
files.download("nutritionist_model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# # load_model = pickle.load(open(filename, 'rb'))rr
# with open("nutritionist_model.pkl","rb") as f:
#   nutritionist_model = pickle.load(f)

In [ ]:
!pip install -q streamlit

In [ ]:
!wget -q -O - ipv4.icanhazip.com


In [ ]:
%%writefile app.py

# Title of app
st.title('AI Nutritionist')

# Adds nutritionist model to app's session state (stores persistent data of app)
#if 'model' not in st.session_state:
  #st.session_state['model'] = 's'

# Adds messages list to app's session state to keep user and chatbot messages
if 'messages' not in st.session_state:
  st.session_state.messages = []

# Enables user input and chatbot responses to be stored in messages
for message in st.session_state.messages:
  with st.chat_message(message['role']):
    st.markdown(message['content'])

# Receives user input
if prompt := st.chat_input('Enter prompt'):
  st.session_state.messages.append({'role': 'user', 'content': prompt}) # Adds input as part of content entered in user part of messages
  with st.chat_message('user'):
    st.markdown(prompt)

  # Generating the model response
  with st.chat_message('dietitian'):
    output = generate_response(st.session_state.messages[-1]['content'])
    response = st.write(output)
    st.session_state.messages.append({'role': 'dietitian', 'content': response})

In [ ]:
! streamlit run app.py & npx localtunnel --port 3000

In [ ]:
!npm install localtunnel

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

In [ ]:
!npx localtunnel --port 3000

In [ ]:
# if False:
#     from unsloth import FastLanguageModel
#     model, tokenizer = FastLanguageModel.from_pretrained(
#         model_name = "nutritionist_model", # YOUR MODEL YOU USED FOR TRAINING
#         max_seq_length = max_seq_length,
#         dtype = dtype,
#         load_in_4bit = load_in_4bit,
#     )
#     FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# # alpaca_prompt = You MUST copy from above!

# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         "I am feeling a severe headache", # instruction
#         "", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
from google.colab import userdata

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("VeronicaObenewaa/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("VeronicaObenewaa/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("VeronicaObenewaa/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "VeronicaObenewaa/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = userdata.get('hugging_face'), # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>